In [7]:
import pandas as pd
import numpy as np
import plotly_express as pe
import matplotlib.pyplot as plt
import folium as fm # map needs a browser to run so will only work in Jupyter
from folium.plugins import FastMarkerCluster # to add map markers
from pyproj import Proj, transform
import matplotlib.pyplot as plt
import time

np.set_printoptions(threshold=np.inf, linewidth = 500, suppress=True)
pd.set_option('display.max_rows', 3000)
pd.set_option('display.max_columns', 3000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [8]:
EURO_COORDINATES = (45.7305, 0)
hotel_reviews = pd.read_csv('Hotel_Reviews.csv') #load hotel data in
MAX_RECORDS = 1000
hotel_reviews.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.700,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.900,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.700,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.500,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.361,4.916
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.700,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.100,"[' Leisure trip ', ' Family with young childre...",3 days,52.361,4.916
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.700,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.800,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.361,4.916
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.700,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.700,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.361,4.916


In [12]:
#Loads in our blank map, zoomed in on Greenpoint, BK
m = fm.Map(location=EURO_COORDINATES, zoom_start=4.5)

m

In [17]:
restaurant_reviews = pd.read_csv('TA_restaurants_curated.csv') #load data in
restaurant_reviews.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.000,5.000,$$ - $$$,136.000,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.000,4.500,$$$$,812.000,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.000,4.500,$$$$,567.000,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.000,5.000,$$$$,564.000,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.000,4.500,$$$$,316.000,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170
